# Visualização de dados

Uma visualização gráfica dos dados auxilia na identificação de possíveis padrões e na tomada de decisões dos pré-processamentos a serem realizados nos dados. Nem sempre é necessário criar gráficos para todo o *dataset*, bastante gráficos suficientes para se ter uma ideia do formato geral dos dados.

Antes de plotar os gráficos é preciso conhecer seus dados. Estaremos utilizando neste `notebook` um *dataset* com dados EMG coletados de dez sujeitos, porém iremos plotar em gráficos apenas os dados de um único sujeito. Neste *dataset* foram coletados 10 movimentos diferentes com dois canais de EMG. Iremos plotar um gráfico para cada canal, apra cada movimento.

Utilizaremos a biblioteca [matplotlib](https://matplotlib.org) para gerar os gráficos.

In [15]:
# Carregando dados
import numpy as np

data = np.load('./lib/data/converted/s1.npy')
print(f'{data.shape} - (classes, ensaios, canais, linhas)')

(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)


In [13]:
# linha necessária para plotar os gráficos no próprio notebook
%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


for movement in range(10):
    plot_data = []
    for trial in range(1):
        for channel in range(1):
            for i, t in enumerate(np.linspace(0, 5, 20000)):
                plot_data.append([channel, t, data[movement][trial][channel][i]])

        plot_data = np.array(plot_data)

        x, y, z = plot_data[:,0], plot_data[:,1], plot_data[:,2]

        fig = plt.figure( figsize=(10, 10), dpi=80)

        ax = fig.add_subplot(1, 1, 1, projection='3d')
        surf = ax.plot_trisurf(x, y, z, cmap=cm.inferno, linewidth=1)
        ax.set_xlabel('Canais')
        ax.set_ylabel('Tempo (seg.)')
        ax.set_zlabel('Milivolts')

        fig.colorbar(surf)
        fig.tight_layout()

        plt.show()

RuntimeError: Error in qhull Delaunay triangulation calculation: input inconsistency (exitcode=1); use python verbose option (-v) to see original qhull error.